# Pseudobulk methods for DE

In [60]:
library(edgeR)

### Repeated

In [171]:
num_groups_per_condition <- 2

data = read.table('/data_volume/memento/simulation/inference/means/pseudobulks_0.csv', sep=',', header=1, row.names=1)
# groups <- c( rep('ctrl', num_groups_per_condition), rep('stim', num_groups_per_condition))
groups <- c('ctrl', 'stim', 'ctrl', 'stim')
names <- 
replicates <- c('A', 'A', 'B', 'B')

y <- DGEList(counts=data,group=groups)
keep <- filterByExpr(y,min.count=1)
y <- y[keep,,keep.lib.sizes=FALSE]
y <- calcNormFactors(y)
design <- model.matrix(~groups + replicates)
y <- estimateDisp(y,design)

fit <- glmFit(y,design)
lrt <- glmLRT(fit,coef=2)
qfit <- glmQLFit(y,design)
qlft <- glmQLFTest(qfit,coef=2)

thresh <- 0.05

fpr <- mean(lrt$table[101:dim(lrt$table)[1],'PValue'] < thresh)

tpr <- mean(lrt$table[1:100, 'PValue'] < thresh)

print(paste('edger','tpr', tpr, 'fpr', fpr, 'at', thresh))

[1] "tpr 0.42 fpr 0.108394768133175 at 0.05"


In [58]:
write.csv(topTags(lrt, n=Inf), '/data_volume/memento/simulation/inference/means/edger_0.csv',)

dispersion_df = data.frame(gene=rownames(y), dispersion=y$tagwise.dispersion)

write.csv(dispersion_df, '/data_volume/memento/simulation/inference/means/dispersions_0.csv')